In [ ]:
import qiskit
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import time
import orjson

from utils import file_to_coupling_map


cmap_name= "heavy_hex_127.txt"
cmap = file_to_coupling_map(f"coupling_maps/{cmap_name}")

results = {}
trial_counts = []

for i in np.linspace(10, 127, num=7, dtype=int):
    results[str(i)] = {'time': [], "depth": [], "swap_count": []}

for trial_count in sorted(set(np.logspace(0, 3, num=20, dtype=int))):
    trial_counts.append(trial_count)
    print(f"trial count {trial_count}")
    for i in np.linspace(10, 127, num=7, dtype=int):
        print(f"qv size {i}")
        qv = qiskit.circuit.library.QFT(num_qubits=i).decompose()
        qv.measure_all()
        sabre_pass = qiskit.transpiler.passes.SabreLayout(cmap, seed=42, max_iterations=4, swap_trials=trial_count, layout_trials=trial_count)
        dag = qiskit.converters.circuit_to_dag(qv)
        start = time.perf_counter()
        out = sabre_pass.run(dag)
        stop = time.perf_counter()
        results[str(i)]["time"].append(stop - start)
        results[str(i)]["depth"].append(out.depth())
        results[str(i)]["swap_count"].append(out.count_ops()["swap"])
        print(stop - start)
        print(out.depth())
        print(out.size())
    with open("output.json", 'wb') as fd:
        fd.write(orjson.dumps(results, option=orjson.OPT_SERIALIZE_NUMPY))

In [ ]:
fig, ax0 = plt.subplots(1)
for i in results:
    print(results[i]["time"])
    ax0.semilogy(trial_counts, results[i]["time"], label=f"{i} qubits")
ax0.set_xlabel("Number of layout and routing trials")
ax0.set_ylabel("Runtime (sec.)")
ax0.set_title("LightSABRE Runtime on QFT Circuits with varied trial counts")
ax0.legend()
fig.tight_layout()
fig.savefig("seed_trials_time.png", dpi=900)

fig, ax1 = plt.subplots(1)
for i in results:
    ax1.plot(trial_counts, results[i]["depth"], label=f"{i} qubits")
ax1.set_xlabel("Number of layout and routing trials")
ax1.set_ylabel("Circuit Depth")
ax1.set_title("LightSABRE Output depth on QFT Circuits with varied trial counts")
ax1.legend()
fig.tight_layout()
fig.savefig("seed_trials_depth.png", dpi=900)

fig, ax2 = plt.subplots(1)
for i in results:
    ax2.plot(trial_counts, results[i]["swap_count"], label=f"{i} qubits")
ax2.set_xlabel("Number of layout and routing trials")
ax2.set_ylabel("Swap count")
ax2.set_title("LightSABRE Output swap_count on QFT Circuits with varied trial counts")
ax2.legend()
fig.tight_layout()
fig.savefig("seed_trials_swap_count.png", dpi=900)
